# Goal

This notebook imports the pyPI code and test data, computes the PI module (pi.py) output, and then performs a few standard analyses on PI data.

In [1]:
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import time
# load PI calculation module
from pi import pi

In [2]:
# change default figure font settings
font = {'family' : 'sans-serif',
        'weight' : 1,
        'size'   : 16}

matplotlib.rc('font', **font)

# Load Data

In [3]:
# data location
_FN='./data/test_data.nc'
# load netcdf file
ds = xr.open_dataset(_FN)

# Calculate PI and Variables

PI is given by the Bister and Emanuel (2002) algorithm (see pi.py):

In [4]:
# create a function to calculate PI over the sample dataset using xarray
def run_sample_dataset(fn, dim='p'):
    """ This is just a flex... """
    
    # open the test data file
    ds = xr.open_dataset(_FN)
    # calculate PI over the whole data set using the xarray universal function
    result = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, miss_handle=1),
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )

    vmax, pmin, ifl, t0, lnb = result

    # return the output from pi.py as an xarray data structure
    return xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'lnb': lnb
    })

In [5]:
# calculate PI over the sample test data
pi_ds=run_sample_dataset(_FN)

## Main development regions (MDRs)

We define the TC main development regions following Gilford et al. (2017):

<img src="figures/mdrs.png" alt="Drawing" style="width: 400px;"/>

In [20]:
# bounds_na=np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2)
# bounds_na

array([[  8.75,  31.25],
       [266.25, 308.75]])

In [21]:
# bounds = xr.Dataset({
#             'na': (
#                 ("lat", "lon"),
#                 np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2),
#                 ),
#             'enp': (
#                 ("lat", "lon"),
#                 np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2),
#                 ),
#             'wnp': (
#                 ("lat", "lon"),
#                 np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2),
#                 ),
#             'ni': (
#                 ("lat", "lon"),
#                 np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2),
#                 ),
#             'sh': (
#                 ("lat", "lon"),
#                 np.asarray([8.75, 31.25, 266.25, 308.75]).reshape(2,2),
#                 ),
#         })

In [28]:
bounds.na

<xarray.DataArray 'na' (lat: 2, lon: 2)>
array([[  8.75,  31.25],
       [266.25, 308.75]])
Dimensions without coordinates: lat, lon

## Compute diagnostics

PI may also be computed directly via:

EQN.

Where...

### Efficiency

The efficiency...
Typical...

### Log PI/Efficiency/Disequilibrium

We decompose each term in this equation by...

# Perform and Plot Analyses

## Annual Climatology

### PI ($V_{max}$)

### Sea Surface Temperature ($T_{s}$)

### Outflow Temperature ($T_0$)

### Outflow Temperature Level (OTL)

### TC Efficiency ($\frac{T_{s} - T_{0}}{T_{0}}$)

### ln[Disequlibrium] ($h^* - h$)

## Seasonal Climatology by Basin